In [1]:
import pygame
import sys

# Initialize pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 600, 600
ROWS, COLS = 4, 4
BOX_SIZE = WIDTH // (COLS + 1)

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)

# Set up the display
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Dot and Boxes")

# Game state
lines = []
boxes = []
player_turn = 1
scores = {1: 0, 2: 0}

# AI Difficulty Levels
EASY, MEDIUM, HARD = 1, 2, 3
ai_difficulty = MEDIUM  # Change this to EASY or HARD for different difficulties

def draw_grid():
    for row in range(ROWS + 1):
        for col in range(COLS + 1):
            pygame.draw.circle(screen, BLACK, (col * BOX_SIZE, row * BOX_SIZE), 5)
            if row < ROWS and col < COLS:
                pygame.draw.rect(screen, WHITE, (col * BOX_SIZE + 5, row * BOX_SIZE + 5, BOX_SIZE - 10, BOX_SIZE - 10))

def draw_lines():
    for line in lines:
        pygame.draw.line(screen, BLUE if line[2] == 1 else RED, line[0], line[1], 5)

def check_boxes():
    global scores
    for row in range(ROWS):
        for col in range(COLS):
            top = ((col * BOX_SIZE, row * BOX_SIZE), ((col + 1) * BOX_SIZE, row * BOX_SIZE))
            right = (((col + 1) * BOX_SIZE, row * BOX_SIZE), ((col + 1) * BOX_SIZE, (row + 1) * BOX_SIZE))
            bottom = ((col * BOX_SIZE, (row + 1) * BOX_SIZE), ((col + 1) * BOX_SIZE, (row + 1) * BOX_SIZE))
            left = ((col * BOX_SIZE, row * BOX_SIZE), (col * BOX_SIZE, (row + 1) * BOX_SIZE))
            if top in lines and right in lines and bottom in lines and left in lines and (row, col) not in boxes:
                boxes.append((row, col))
                scores[player_turn] += 1

def draw_boxes():
    for box in boxes:
        row, col = box
        pygame.draw.rect(screen, BLUE if player_turn == 1 else RED, (col * BOX_SIZE + 5, row * BOX_SIZE + 5, BOX_SIZE - 10, BOX_SIZE - 10))

def ai_move():
    global player_turn
    if ai_difficulty == EASY:
        easy_ai_move()
    elif ai_difficulty == MEDIUM:
        medium_ai_move()
    else:
        hard_ai_move()
    player_turn = 1 if player_turn == 2 else 2

def easy_ai_move():
    for row in range(ROWS):
        for col in range(COLS):
            possible_moves = [
                ((col * BOX_SIZE, row * BOX_SIZE), ((col + 1) * BOX_SIZE, row * BOX_SIZE)),
                (((col + 1) * BOX_SIZE, row * BOX_SIZE), ((col + 1) * BOX_SIZE, (row + 1) * BOX_SIZE)),
                ((col * BOX_SIZE, (row + 1) * BOX_SIZE), ((col + 1) * BOX_SIZE, (row + 1) * BOX_SIZE)),
                ((col * BOX_SIZE, row * BOX_SIZE), (col * BOX_SIZE, (row + 1) * BOX_SIZE))
            ]
            for move in possible_moves:
                if move not in lines and (move[1], move[0]) not in lines:
                    lines.append((*move, 2))
                    return

def medium_ai_move():
    # AI tries to complete a box
    for row in range(ROWS):
        for col in range(COLS):
            top = ((col * BOX_SIZE, row * BOX_SIZE), ((col + 1) * BOX_SIZE, row * BOX_SIZE))
            right = (((col + 1) * BOX_SIZE, row * BOX_SIZE), ((col + 1) * BOX_SIZE, (row + 1) * BOX_SIZE))
            bottom = ((col * BOX_SIZE, (row + 1) * BOX_SIZE), ((col + 1) * BOX_SIZE, (row + 1) * BOX_SIZE))
            left = ((col * BOX_SIZE, row * BOX_SIZE), (col * BOX_SIZE, (row + 1) * BOX_SIZE))
            sides = [top, right, bottom, left]
            filled_sides = [side for side in sides if side in lines or (side[1], side[0]) in lines]
            if len(filled_sides) == 3:
                for side in sides:
                    if side not in lines and (side[1], side[0]) not in lines:
                        lines.append((*side, 2))
                        return
    easy_ai_move()

def hard_ai_move():
    # AI avoids creating a box for the opponent
    for row in range(ROWS):
        for col in range(COLS):
            top = ((col * BOX_SIZE, row * BOX_SIZE), ((col + 1) * BOX_SIZE, row * BOX_SIZE))
            right = (((col + 1) * BOX_SIZE, row * BOX_SIZE), ((col + 1) * BOX_SIZE, (row + 1) * BOX_SIZE))
            bottom = ((col * BOX_SIZE, (row + 1) * BOX_SIZE), ((col + 1) * BOX_SIZE, (row + 1) * BOX_SIZE))
            left = ((col * BOX_SIZE, row * BOX_SIZE), (col * BOX_SIZE, (row + 1) * BOX_SIZE))
            sides = [top, right, bottom, left]
            filled_sides = [side for side in sides if side in lines or (side[1], side[0]) in lines]
            if len(filled_sides) == 3:
                for side in sides:
                    if side not in lines and (side[1], side[0]) not in lines:
                        lines.append((*side, 2))
                        return
    for row in range(ROWS):
        for col in range(COLS):
            top = ((col * BOX_SIZE, row * BOX_SIZE), ((col + 1) * BOX_SIZE, row * BOX_SIZE))
            right = (((col + 1) * BOX_SIZE, row * BOX_SIZE), ((col + 1) * BOX_SIZE, (row + 1) * BOX_SIZE))
            bottom = ((col * BOX_SIZE, (row + 1) * BOX_SIZE), ((col + 1) * BOX_SIZE, (row + 1) * BOX_SIZE))
            left = ((col * BOX_SIZE, row * BOX_SIZE), (col * BOX_SIZE, (row + 1) * BOX_SIZE))
            sides = [top, right, bottom, left]
            filled_sides = [side for side in sides if side in lines or (side[1], side[0]) in lines]
            if len(filled_sides) == 2:
                for side in sides:
                    if side not in lines and (side[1], side[0]) not in lines:
                        lines.append((*side, 2))
                        return
    easy_ai_move()

# Main game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                # Reset the game
                lines.clear()
                boxes.clear()
                scores = {1: 0, 2: 0}
                player_turn = 1
            if event.key == pygame.K_q or event.key == pygame.K_ESCAPE:
                running = False

    if player_turn == 2:
        ai_move()
    else:
        # Handle player input (e.g., mouse clicks or key presses)
        pass  # Placeholder for player input handling

    check_boxes()

    screen.fill(WHITE)
    draw_grid()
    draw_lines()
    draw_boxes()
    pygame.display.flip()

pygame.quit()
sys.exit()


ModuleNotFoundError: No module named 'pygame'